# Notebook for Debugging BACE Application Locally

To run your API locally and debug the application:

* Run `sam build` to create the current version of your local application.
* Run `sam local start-api` from the root directory of your application.

This creates a local host for you application that you can access by default at `http://localhost:3000/`.

* Note: 
  * if you do not exit the `sam local start-api` call correctly (the command line will show you the right exit command: `Press CTRL+C to quit`), then you'll get the error message `Port 3000 is in use by another program.` the next time you run `sam local start-api`.
  * In that case, on Mac/Linux, you can clear the stale call from the command line as follows:
  
    ```bash
        lsof -i tcp:3000
        kill -9 PID
    ```
    The first command locates the PID of the existing `sam` command call to port 3000. Use that PID in the second command.

Use this notebook to test API calls locally. This can be particularly helpful for debugging the `app/bace/user_convert.py` functions.

When you make changes to your local files, make sure to rerun `sam build` so that these changes are reflected in your local application.

Please note that the observed speeds from testing the application locally may not accurately reflect the speeds you will experience when the application is deployed on the cloud.

In [1]:
# Preliminaries
import json
import requests
import time
from IPython.display import display, HTML


base_url = "http://localhost:3000"
print(f'URL for your API\n{base_url}')

URL for your API
http://localhost:3000


# Homepage

GET Request to test that the application is up and running

In [2]:
print('Query local URL homepage.')
print('Note: Local queries do not accurately capture the speed from querying the application when it is hosted on AWS.')

# First Call
t0 = time.time()
route = "/"

try:
    homepage = requests.get(
        f'{base_url}{route}'
    )
    print('Response:')
    print(json.loads(homepage.text))
    print(f'Time to receive call: {time.time() - t0}s\n')
except:
    print('Make sure that you run `sam build` and `sam local start-api` from the command line to host your application locally.')


Query local URL homepage.
Note: Local queries do not accurately capture the speed from querying the application when it is hosted on AWS.
Response:
{'message': 'Hello! Your BACE application is up and running.', 'author': 'Pen Example Application'}
Time to receive call: 2.9229278564453125s



# Random Design

GET request to receive a random design

In [3]:
route = "/random_design"
design = requests.get(
    f'{base_url}{route}'
)

print(json.loads(design.text))

{'color_a': 'Blue', 'color_b': 'Blue', 'price_a': 5.300117943752238, 'price_b': 0.576921695412509, 'type_a': 'Ballpoint', 'type_b': 'Gel'}


# Route: /create_profile

API request to create a new profile for an individual.

In [4]:
route = "/create_profile"

data = {
    "survey_id": "test_colab",
    # Include additional variables that you want stored with an individual's profile here.
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)
profile_id = r.get('profile_id')

print(r)

try:
    display(HTML(r['message_0_1']))
    display(HTML(r['message_1_1']))
except:
    print(r)

{'profile_id': '689d2a5c-a875-4661-854f-35d9573426be', 'color_a_1': 'Black', 'color_b_1': 'Blue', 'price_a_1': 3.986736670584932, 'price_b_1': 3.965608415582622, 'type_a_1': 'Gel', 'type_b_1': 'Gel', 'message_0_1': '\n        <table width=\'300px\' border=\'1\' cellpadding=\'1\' cellspacing=\'1\' style=\'font-family: Arial, Tahoma, "Helvetica Neue", Helvetica, sans-serif; border-collapse:collapse; background-color:#eee9e7; color:black;\'>\n            <tbody>\n                <tr>\n                    <th style="text-align: center; background-color: #ded4ce;"><b>Pen A</b></th>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Price:</em><br> $3.99</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Color:</em><br> Black</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Type:</em><br> Gel</td>\n                </tr>\n

Pen A
Price: $3.99
Pen Color: Black
Pen Type: Gel


Pen B
Price: $3.97
Pen Color: Blue
Pen Type: Gel


# Route /update_profile

API request to update profile for an individual

In [5]:
route = "/update_profile"

##### Answer to previous question ########
answer = 1
##########################################

data = {
    "profile_id": profile_id,
    "answer": answer,
    # Include additional variables you want in the body of the request here
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)

print(r)

question_number = list(r.keys())[0].rpartition('_')[-1]

try:
    display(HTML(r[f'message_0_{question_number}']))
    display(HTML(r[f'message_1_{question_number}']))
except:
    print(r)

{'color_a_2': 'Blue', 'color_b_2': 'Black', 'price_a_2': 5.264922325989443, 'price_b_2': 3.9935457400110344, 'type_a_2': 'Gel', 'type_b_2': 'Ballpoint', 'message_0_2': '\n        <table width=\'300px\' border=\'1\' cellpadding=\'1\' cellspacing=\'1\' style=\'font-family: Arial, Tahoma, "Helvetica Neue", Helvetica, sans-serif; border-collapse:collapse; background-color:#eee9e7; color:black;\'>\n            <tbody>\n                <tr>\n                    <th style="text-align: center; background-color: #ded4ce;"><b>Pen A</b></th>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Price:</em><br> $5.26</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Color:</em><br> Blue</td>\n                </tr>\n                <tr>\n                    <td style="text-align: center;"><em>Pen Type:</em><br> Gel</td>\n                </tr>\n            </tbody>\n        </table>\n    ', '

Pen A
Price: $5.26
Pen Color: Blue
Pen Type: Gel


Pen B
Price: $3.99
Pen Color: Black
Pen Type: Ballpoint


# Posterior Estimates

Route: /estimates

## POST
API request to update profile with a given answer and return estimates

In [6]:
route = "/estimates"
data = {
    "profile_id": profile_id,
    "answer": '1'
}

r = requests.post(
    f'{base_url}{route}',
    data=data
)

r = json.loads(r.text)

print(r)

{'blue_ink': {'mean': 0.4901820336979743, 'median': 0.47102823611691447, 'std': 0.6802569099097395}, 'gel_pen': {'mean': 0.122504593961262, 'median': 0.13201784078931866, 'std': 0.7826118657866036}, 'mu': {'mean': 5.235745459706961, 'median': 5.176212405345765, 'std': 2.666914803484577}}


## GET
API request toreturn estimates for a given profile

In [7]:
route = "/estimates"
params = {
    "profile_id": profile_id,
}

r = requests.get(
    f'{base_url}{route}',
    params=params
)

r = json.loads(r.text)

print(r)

{'mu': {'median': '5.176212405345765', 'mean': '5.235745459706961', 'std': '2.666914803484577'}, 'blue_ink': {'median': '0.47102823611691447', 'mean': '0.4901820336979743', 'std': '0.6802569099097395'}, 'gel_pen': {'median': '0.13201784078931866', 'mean': '0.122504593961262', 'std': '0.7826118657866036'}}
